# Map Creation

This map integration uses the existing databases provided by Madrid's city council. This include information of traffic lights, lamps and CCTV trafic cameras. In this notebook analyzes the datasets cleaning them, simplifying them for the info we need and constructs and HTML interactive map to visualize and count the number of entities in specific territorial areas of interest. Specifically Salamanca, Retiro and Centro. 

In [76]:
#importing libraries
import pandas as pd
import numpy as np
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import re
from pyproj import Transformer

## CCTV Dataset

This dataset contains information about fixed traffic cameras installed in the city of Madrid. The primary purpose of these cameras is to monitor real-time traffic conditions and any events that may affect mobility. This enables authorities to implement appropriate measures for traffic regulation. The dataset includes a KML file that provides the geographic location of each camera, which has been converted into a CSV format for easier processing and analysis. 

Additionally, the dataset includes images captured by the cameras, updated every five minutes. Associated documentation is provided to help users interpret the dataset fields correctly, along with usage recommendations and additional information for better understanding. 

The dataset can be accessed and visualized alongside other traffic information for Madrid on the [Informo Portal](https://informo.madrid.es). 
Direct access to the dataset is available here: [Madrid Open Data Portal](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=8803c23866b93410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default).

In [77]:
# importing the dataset
cctv = pd.read_csv('CCTV.csv')

# removing the columns we don't need (Unnamed:4) and (Z)
cctv = cctv.drop(columns=['Unnamed: 4', 'Z'])

# Function to extract image URL and description
def extract_image_and_description(html):
    # Extract image URL
    image_match = re.search(r'src=(https://[^\s]+)', html)
    image_url = image_match.group(1) if image_match else None
    
    # Extract description
    description_match = re.search(r'<br/>(.*?)</div>', html)
    description = description_match.group(1).strip() if description_match else None
    
    return pd.Series([image_url, description])

# Apply the function to the description column
cctv[['image', 'location']] = cctv['description'].apply(extract_image_and_description)

# removing the description column
cctv = cctv.drop(columns=['description'])

# renaming X -> Longitude and Y -> Latitude
cctv = cctv.rename(columns={'X': 'Longitude', 'Y': 'Latitude'})

# visualizing the dataset
display(cctv.head())




,Longitude,Latitude,image,location
0,-3.688942,40.466064,https://informo.madrid.es/cameras/Camara06303....,PLAZA DE CASTILLA (NORTE)
1,-3.689632,40.465776,https://informo.madrid.es/cameras/Camara06304....,PLAZA DE CASTILLA (SUR)
2,-3.685597,40.458951,https://informo.madrid.es/cameras/Camara05325....,ALBERTO ALCOCER-PADRE DAMIAN
3,-3.690489,40.452028,https://informo.madrid.es/cameras/Camara06306....,PLAZA DE LIMA
4,-3.704070,40.446991,https://informo.madrid.es/cameras/Camara06308....,CUATRO CAMINOS


## LAMPS Dataset

This dataset contains information about luminous units (streetlights) in the city of Madrid. Key details include:
- A luminous unit is any structure that provides light, regardless of the number of lamps or LED elements it contains.  
  Example: A pole with a sodium lamp for the roadway and a globe light for pedestrians is considered a single unit.
- Luminous units are categorized into three types:
  - **LED**: All lamps in the unit are LED.
  - **DESCARGA**: All lamps are discharge-based (e.g., incandescent, halogen).
  - **LED-DESCARGA**: A mix of LED and discharge lamps in the same unit (common in the city center).
- Each unit is geolocated using the municipal street map.
- Additional details include the specific model of the luminaire ("tipo_bloq").
- Note: This dataset does not include luminous units from parks, gardens, or the M-30 highway.
- The dataset can be accessed and downloaded from the [Madrid Open Data Portal](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=72b76cc09a800810VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default).

In [79]:
# import dataset
lamps = pd.read_excel('LAMPS.xlsx')

# visualizing the dataset
print("Before cleaning / Original dataset")
display(lamps.head())

# drop the columns we don't need (tipo_bloqu,C,50) and (CLASE_APP,C,254) and (COD_NDP,N,10,0)
lamps = lamps.drop(columns=['tipo_bloqu,C,50', 'CLASE_APP,C,254', 'COD_NDP,N,10,0'])

# renaming the columns

# (TIPOL,C,254) to (type)
lamps.rename(columns={'TIPOL,C,254':'type'}, inplace=True)
# (VIA_CLASE,C,254) to (type_of_road)
lamps.rename(columns={'VIA_CLASE,C,254':'type_of_road'}, inplace=True)
# (VIA_PAR,C,254) to (side)
lamps.rename(columns={'VIA_PAR,C,254':'side'}, inplace=True)
# (VIA_NOMB,C,254) to (street_name)
lamps.rename(columns={'VIA_NOMBRE,C,254':'street_name'}, inplace=True)
# (NUM_POSTE,N,10,0) to (stret_number)
lamps.rename(columns={'NUMERO,N,10,0':'street_number'}, inplace=True)
# (DISTIRITO,N,10,0) to (district)
lamps.rename(columns={'DISTRITO,N,10,0':'district'}, inplace=True)
# (BATTIO,N,10,0) to (neighborhood)
lamps.rename(columns={'BARRIO,N,10,0':'neighborhood'}, inplace=True)
# (X_UTM,N,19,11) to (x_utm)
lamps.rename(columns={'X_UTM,N,19,11':'x_utm'}, inplace=True)
# (Y_UTM,N,19,11) to (y_utm)
lamps.rename(columns={'Y_UTM,N,19,11':'y_utm'}, inplace=True)

# transforming x_utm and y_utm transformer = Transformer.from_crs("EPSG:32630", "EPSG:4326")
transformer = Transformer.from_crs("EPSG:32630", "EPSG:4326")
lamps['Latitude'], lamps['Longitude'] = transformer.transform(
    lamps["x_utm"].values, 
    lamps["y_utm"].values
)

# dropping the x_utm and y_utm columns
lamps = lamps.drop(columns=['x_utm', 'y_utm'])

# districts dictionary

districts = {
    1: 'Centro',
    2: 'Arganzuela',
    3: 'Retiro',
    4: 'Salamanca',
    5: 'Chamartín',
    6: 'Tetuán',
    7: 'Chamberí',
    8: 'Fuencarral-El Pardo',
    9: 'Moncloa',
    10: 'Latina',
    11: 'Carabanchel',
    12: 'Usera',
    13: 'Puente de Vallecas',
    14: 'Moratalaz',
    15: 'Ciudad Lineal',
    16: 'Hortaleza',
    17: 'Villaverde',
    18: 'Villa de Vallecas',
    19: 'Vicálvaro',
    20: 'San Blas',
    21: 'Barajas'
}

# neighborhoods dictionary

neighborhoods = {
    1: ['Palacio', 'Embajadores', 'Cortes', 'Justicia', 'Universidad', 'Sol'],  # Centro (6 neighborhoods)
    2: ['Imperial', 'Acacias', 'Chopera', 'Legazpi', 'Delicias', 'Palos de la Frontera', 'Atocha'],  # Arganzuela (7 neighborhoods)
    3: ['Pacífico', 'Adelfas', 'Estrella', 'Ibiza', 'Jerónimos', 'Niño Jesús'],  # Retiro (6 neighborhoods)
    4: ['Recoletos', 'Goya', 'Fuente del Berro', 'Guindalera', 'Lista', 'Castellana'],  # Salamanca (6 neighborhoods)
    5: ['El Viso', 'Prosperidad', 'Ciudad Jardín', 'Hispanoamérica', 'Nueva España', 'Castilla'],  # Chamartín (6 neighborhoods)
    6: ['Bellas Vistas', 'Cuatro Caminos', 'Castillejos', 'Almenara', 'Valdeacederas', 'Berruguete'],  # Tetuán (6 neighborhoods)
    7: ['Gaztambide', 'Arapiles', 'Trafalgar', 'Almagro', 'Ríos Rosas', 'Vallehermoso'],  # Chamberí (6 neighborhoods)
    8: ['El Pardo', 'Fuentelarreina', 'Peñagrande', 'Pilar', 'La Paz', 'Valverde', 'Mirasierra', 'El Goloso'],  # Fuencarral-El Pardo (8 neighborhoods)
    9: ['Casa de Campo', 'Argüelles', 'Ciudad Universitaria', 'Valdezarza', 'Valdemarín', 'El Plantío', 'Aravaca'],  # Moncloa-Aravaca (7 neighborhoods)
    10: ['Los Cármenes', 'Puerta del Ángel', 'Lucero', 'Aluche', 'Campamento', 'Cuatro Vientos', 'Las Águilas'],  # Latina (7 neighborhoods)
    11: ['Comillas', 'Opañel', 'San Isidro', 'Vista Alegre', 'Puerta Bonita', 'Buenavista', 'Abrantes'],  # Carabanchel (7 neighborhoods)
    12: ['Orcasitas', 'Orcasur', 'San Fermín', 'Almendrales', 'Moscardó', 'Zofío', 'Pradolongo'],  # Usera (7 neighborhoods)
    13: ['Entrevías', 'San Diego', 'Palomeras Bajas', 'Palomeras Sureste', 'Portazgo', 'Numancia'],  # Puente de Vallecas (6 neighborhoods)
    14: ['Pavones', 'Horcajo', 'Marroquina', 'Media Legua', 'Fontarrón', 'Vinateros'],  # Moratalaz (6 neighborhoods)
    15: ['Ventas', 'Pueblo Nuevo', 'Quintana', 'Concepción', 'San Pascual', 'San Juan Bautista', 'Colina', 'Atalaya', 'Costillares'],  # Ciudad Lineal (9 neighborhoods)
    16: ['Palomas', 'Piovera', 'Canillas', 'Pinar del Rey', 'Apóstol Santiago', 'Valdefuentes'],  # Hortaleza (6 neighborhoods)
    17: ['Villaverde Alto', 'San Cristóbal', 'Butarque', 'Los Rosales', 'Los Ángeles'],  # Villaverde (5 neighborhoods)
    18: ['Casco Histórico de Vallecas', 'Santa Eugenia', 'Ensanche de Vallecas'],  # Villa de Vallecas (3 neighborhoods)
    19: ['Casco Histórico de Vicálvaro', 'Valdebernardo', 'Valderrivas', 'El Cañaveral'],  # Vicálvaro (4 neighborhoods)
    20: ['Simancas', 'Hellín', 'Amposta', 'Arcos', 'Rosas', 'Rejas', 'Canillejas', 'Salvador'],  # San Blas-Canillejas (8 neighborhoods)
    21: ['Alameda de Osuna', 'Aeropuerto', 'Casco Histórico de Barajas', 'Timón', 'Corralejos']  # Barajas (5 neighborhoods)
}

# Replace district numbers with names
lamps['district'] = lamps['district'].map(districts)

# Replace neighborhood numbers with names
lamps['neighborhood'] = lamps.apply(lambda row: neighborhoods[list(districts.keys())[list(districts.values()).index(row['district'])]][int(row['neighborhood']) - 1], axis=1)

# making type_of_road, side, street_name and street_number into just one column, address
lamps['address'] = lamps['type_of_road'] + ' ' + lamps['side'] + ' ' + lamps['street_name'] + ', ' + lamps['street_number'].astype(str)

# dropping the columns we don't need anymore
lamps = lamps.drop(columns=['type_of_road', 'side', 'street_name', 'street_number'])

# visualizing the dataset
print("After cleaning the dataset and transforming the columns")
display(lamps.head())

Before cleaning / Original dataset


,"tipo_bloqu,C,50","TIPOL,C,254","VIA_CLASE,C,254","VIA_PAR,C,254","VIA_NOMBRE,C,254","CLASE_APP,C,254","NUMERO,N,10,0","COD_NDP,N,10,0","DISTRITO,N,10,0","BARRIO,N,10,0","X_UTM,N,19,11","Y_UTM,N,19,11"
0,LBE002,DESCARGA,CALLE,DE LAS,ERAS,NUMERO,3,20040016,16,4,445648.4812,4.480671e+06
1,LBE002,DESCARGA,CALLE,DE LAS,ERAS,NUMERO,10,31024246,16,4,445610.6416,4.480741e+06
2,GLED001,LED,PLAZA,DE,MAR DEL PLATA,NUMERO,12,11119324,16,4,445680.0750,4.480679e+06
3,FFLED005,LED,CALLE,DEL,MAR AMARILLO,NUMERO,21,11119327,16,4,445669.5353,4.480645e+06
4,FFLED005,LED,CALLE,DEL,MAR AMARILLO,NUMERO,19,11119326,16,4,445651.1627,4.480635e+06


After cleaning the dataset and transforming the columns


,type,district,neighborhood,Latitude,Longitude,address
0,DESCARGA,Hortaleza,Pinar del Rey,40.474942,-3.641197,"CALLE DE LAS ERAS, 3"
1,DESCARGA,Hortaleza,Pinar del Rey,40.475573,-3.641650,"CALLE DE LAS ERAS, 10"
2,LED,Hortaleza,Pinar del Rey,40.475020,-3.640825,"PLAZA DE MAR DEL PLATA, 12"
3,LED,Hortaleza,Pinar del Rey,40.474709,-3.640947,"CALLE DEL MAR AMARILLO, 21"
4,LED,Hortaleza,Pinar del Rey,40.474621,-3.641163,"CALLE DEL MAR AMARILLO, 19"


## CRUCES Dataset

This dataset contains the **location of traffic signal intersections** in the city of Madrid, along with the date of their initial installation. In addition to its intrinsic value, this dataset allows for the association of other devices and elements from related datasets, such as:
- **Traffic Light Heads**: Information about the heads of traffic lights.
- **Traffic Lights with Acoustic Signals**: Traffic lights equipped with auditory signals.
- **Bicycle Traffic Signal Intersections**: Traffic signal intersections specifically designed for bicycles.

The dataset can be accessed and downloaded from the [Madrid Open Data Portal](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=2ade509d10786610VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default).

In [80]:
# importing the dataset with specified encoding and handling bad lines
cruces = pd.read_csv('CRUCES.csv', encoding='latin1', sep=';', on_bad_lines='skip')

# dropping the columns we don't need (id, fecha_inst,utm_x,utm_y)
cruces = cruces.drop(columns=['fecha_inst', 'utm_x', 'utm_y'])

# renaming distrito to district, descripcion to description, tipo to type, ubicacion to location, longitud for Longitude and latitud for Latitude
cruces.rename(columns={'distrito': 'district', 'descripcion': 'description', 'tipo': 'type', 'ubicacion': 'location', 'longitud': 'Longitude', 'latitud': 'Latitude'}, inplace=True)

# changing the number of every district to its name
cruces['district'] = cruces['district'].map(districts)

display(cruces.head())

,district,id,description,Longitude,Latitude
0,Hortaleza,4749,AV. FUERZAS ARMADAS 116 P.P.,-3.632115,40.485992
1,San Blas,2165,AV. GUADALAJARA - AUSTRIA,-3.608130,40.423932
2,San Blas,6,SAN HILARIO - AV. LUIS ARAGONES,-3.605220,40.446228
3,San Blas,1975,NIZA 53 - P.P. - TRAVESIA RONDA,-3.607504,40.432102
4,Tetuán,4690,Pº CASTELLANA 89-93,-3.691144,40.450433


### TRAFIC_LIGHTS_HEADS

This dataset provides information about **traffic light heads** installed in the city of Madrid. Traffic light heads are the components that house the lights of a traffic signal system. They are used to regulate and signal the movement of pedestrians, bicycles, and motor vehicles, and can consist of one or more lights. Key details include:
- All traffic light heads currently use **LED systems** due to their numerous advantages, such as low energy consumption, minimal maintenance, environmental friendliness, and resistance to complete failure (as they are composed of a matrix of diodes). Additionally, they are protected against UV radiation.
- The dataset includes technical details about their usage, dimensions, and the number of lights.
- Associated documentation is provided to help interpret the various fields in the dataset, along with usage recommendations and additional information for better understanding.

The dataset can be accessed and downloaded from the [Madrid Open Data Portal](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=1e69ee425d7c6410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default).

In [81]:
# importing the dataset
trafic = pd.read_excel('TRAFIC_LIGHTS_HEADS.xlsx')

# rename longitude and latitude columns to Longitude and Latitude, tipo to type, distrito to district
trafic.rename(columns={'longitud':'Longitude', 'latitud':'Latitude', 'tipo':'type', 'distrito':'district'}, inplace=True)

# droping utm_x, utm_y, fecha_inst
trafic = trafic.drop(columns=['utm_x', 'utm_y', 'fecha_inst'])

# changing the number of every district to its name
trafic['district'] = trafic['district'].map(districts)

display(trafic.head())

,tipo_elem,district,id,id_cruce,Longitude,Latitude
0,SEMAFORO 2/200 PEATON,Centro,37203,99,-3.705259,40.429605
1,SEMAFORO 2/100,Centro,41153,99,-3.706057,40.429034
2,SEMAFORO 2/200 PEATON,Centro,37211,99,-3.705837,40.428900
3,SEMAFORO 3/200,Centro,19750,99,-3.705690,40.429576
4,SEMAFORO 2/200 PEATON,Centro,37212,99,-3.705839,40.428891


### TRAFIC_LIGHTS_ACUSTIC

This dataset provides information about the **characteristics and location of acoustic traffic signals** installed in the city of Madrid. These devices are essential for ensuring accessibility, as public entities are required by current regulations to facilitate the use of traffic signal intersections through acoustic devices. This allows individuals with visual impairments or other accessibility needs to safely use pedestrian crossings at signalized intersections.

Key details include:
- Acoustic signals operate continuously from **8:00 AM to 10:00 PM daily**, balancing the right to autonomous mobility for visually impaired individuals and the right to rest for all citizens.
- Madrid currently has over **9,000 installed devices**, most of which use digital technology. This allows for features such as volume regulation, the use of different sounds, and time-based programming.
- Associated documentation is provided to help interpret the various fields in the dataset, along with usage recommendations and additional information for better understanding.

This information can be visualized, along with other traffic-related data for Madrid, on the [Informo Portal](https://informo.madrid.es).

In [82]:
# Importing the dataset
acustic = pd.read_csv('TRAFIC_LIGHTS_ACUSTIC.csv', sep=';')

# dropping the columns we don't need (fecha_inst, utm_x, utm_y)
acustic = acustic.drop(columns=['fecha_inst', 'utm_x', 'utm_y'])

#change the columns names longitud- > Longitude, latitud -> Latitude, distrito -> district, tipo_elem -> type
acustic.rename(columns={'longitud':'Longitude', 'latitud':'Latitude', 'distrito':'district', 'tipo_elem':'type'}, inplace=True)

# changing the number of every district to its name
acustic['district'] = acustic['district'].map(districts)

# visualizing the dataset
display(acustic.head())

,type,district,id,id_cruce,Longitude,Latitude
0,AVISADOR ACUSTICO (MODERNO - DIGITAL PROGRAMAB...,Centro,3442,99,-3.705518,40.430051
1,AVISADOR ACUSTICO (MODERNO - DIGITAL PROGRAMAB...,Centro,3444,99,-3.705245,40.429411
2,AVISADOR ACUSTICO (MODERNO - DIGITAL PROGRAMAB...,Centro,3439,99,-3.706356,40.429531
3,AVISADOR ACUSTICO (MODERNO - DIGITAL PROGRAMAB...,Centro,3443,99,-3.705267,40.429625
4,AVISADOR ACUSTICO (MODERNO - DIGITAL PROGRAMAB...,Centro,3445,99,-3.705836,40.429176


# Downloading the updated datasets

In [83]:
# downloading the updated datasets in a new folder called datasets:
cctv.to_csv('datasets/cctv.csv', index=False)
lamps.to_csv('datasets/lamps.csv', index=False)
cruces.to_csv('datasets/cruces.csv', index=False)
trafic.to_csv('datasets/trafic.csv', index=False)
acustic.to_csv('datasets/acustic.csv', index=False)